In [ ]:
import timm
import torch


def get_features(dataset, batch, num_images):
    if torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'

    model = timm.create_model('resnet50', num_classes=4)
    model = model.load_state_dict(torch.load('C:\\Users\\rtm\Desktop\\tsne_new\\best.pt'))
    model.eval()
    model.to(device)

    dataset = MyDataset(dataset, num_images)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch, collate_fn=collate_skip_empty, shuffle=True)

    # feature 저장 변수
    features = None

    # label 저장, image_path 저장
    labels = []
    image_paths = []

    for batch in tqdm(dataloader, desc='Running the model inference'):
        images = batch['image'].to(device)
        labels += batch['label']
        image_paths += batch['image_path']

        with torch.no_grad():
            output = model.forward_features(images)

        current_features = output.cpu().numpy()
        if features is not None:
            features = np.concatenate((features, current_features))
        else:
            features = current_features

    return features, labels, image_paths
1:44
def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('--path', type=str, default='data')
    parser.add_argument('--batch', type=int, default=64)
    parser.add_argument('--num_images', type=int, default=500)
    args = parser.parse_args()

    fix_random_seeds()

    features, labels, image_paths = get_features(
        dataset=args.path,
        batch=args.batch,
        num_images=args.num_images
    )
    print(features.shape)

if __name__ == '__main__':
    main()